##**10. Text Generation**
1. Pretrained Model을 이용해 Text를 generation 하는 model을 구현합니다.
2. 실제 데이터셋을 가지고 모델을 학습해봅니다.
3. 다양한 decoding strategy를 이용하여 text를 생성해봅니다.

In [28]:
!pip install transformers==4.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

In [30]:
import numpy as np
import random

def set_seed(random_seed):
    torch.random.manual_seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)


In [31]:
set_seed(777)

In [32]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [33]:
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]), 
                           max_length=50,repetition_penalty=1.0, #no penalty
                           top_k=5, #5
                           temperature=1.0,                          
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id)

In [34]:
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민, 무기질 등 영양소가 풍부한 음식을 골고루 섭취하는 것이 좋다.
또한 규칙적인 운동은 근육을 강화시켜주는 효과가 있다.
특히, 아침식사는 단백질과 비타민


In [35]:
print(gen_ids[0,:])
print(tokenizer.decode(gen_ids[0,:]))

tensor([33245, 10114, 12748, 11357, 23879, 39306,  9684,  7884, 10211, 15177,
        26421,   387, 17339,  7889,  9908, 15768,  6903, 15386,   387, 13830,
         8267,  9030, 12443, 10386, 12923, 16285, 31717, 17764,  9033,  9199,
        14441,  7335,  8704, 39306, 19295, 14222,  8137, 10416, 10108, 10090,
        14830,  9029, 26421,   387, 17339,  7889,  9908, 15768,  6903, 15386])
근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민, 무기질 등 영양소가 풍부한 음식을 골고루 섭취하는 것이 좋다.
또한 규칙적인 운동은 근육을 강화시켜주는 효과가 있다.
특히, 아침식사는 단백질과 비타민


## Likelihood-based Decoding

* Greedy Search
* Beam Search

In [36]:
def greedy(logits):
    return torch.argmax(logits, dim=-1, keepdim=True)

In [37]:
class SamplerBase:
    def __init__(self, model, seq_length):
        self.model = model
        self.seq_length = seq_length

    def sample(self, inps, past):
        return NotImplementedError

In [38]:
from copy import deepcopy

## Greedy Search Decoding

In [39]:

def greedy(logits):
    return torch.argmax(logits, dim=-1, keepdim=True)

class GreedySampler(SamplerBase):
    def __init__(self, model, seq_length, top_whatever, stochastic=False, temperature: float = 1.0):
        """
        :param model:
        :param seq_length:
        :param stochastic: choice [top_k,top_p] if True
        """
        super(GreedySampler, self).__init__(model, seq_length)

        self.sampling = greedy

    @torch.no_grad()
    def sample(self, inps):
        inps=torch.LongTensor([inps])        
        context = inps
        generated = deepcopy(inps)
        past = None

        for t in range(0, self.seq_length):
            out = self.model(context, past_key_values=past)
            lm_logits,past= out["logits"],out["past_key_values"]
            
            lm_logits = lm_logits[:, -1]
            
            context = self.sampling(lm_logits)
            generated = torch.cat([generated, context], dim=-1)

        return generated


## Hugging face Library

In [40]:
gen_ids = model.generate(torch.tensor([input_ids]),max_length=34)

generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민, 무기질 등 영양소가 풍부한 음식을 골고루 섭취하는 것이 좋다.
또한 규칙적인


## 비교해보기

In [41]:

sampler=GreedySampler(model,30,1)

sampled_ids=sampler.sample(input_ids)

generated = tokenizer.decode(sampled_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민, 무기질 등 영양소가 풍부한 음식을 골고루 섭취하는 것이 좋다.
또한 규칙적인


## Beam Search Decoding

In [42]:
class BeamSampler(SamplerBase):
    def __init__(self, model, seq_length, beam_size: int = 3, temperature: float = 1.0):
        """
        no version on stochastic mode
        :param model:
        :param seq_length:
        :param top_whatever: int as beam_size
        """
        super(BeamSampler, self).__init__(model, seq_length)
        self.temperature = temperature
        # if not isinstance(beam_size, int):
        #     raise ValueError
        self.beam_size = beam_size
        self.sampling = greedy

    def _set_start_sequence(self, inps):
        batch, seq_lens = inps.size()
        res = inps[:, None].repeat(1, self.beam_size, 1)  # [batch, beam, l]
        res.view(-1, seq_lens)

        return res.view(-1, seq_lens)

    @torch.no_grad()
    def sample(self, inps):
        inps=torch.LongTensor([inps])
        n_batch, seq_length = inps.size()
        context = self._set_start_sequence(inps)
        generated = deepcopy(context)
        past = None

        probs = torch.zeros([n_batch * self.beam_size]).to(context.device)
        for t in range(0, self.seq_length):
            out = self.model(context, past_key_values=past)
            lm_logits,past= out["logits"],out["past_key_values"]
#             lm_logits, past = self.model(context, past=past)
            
            lm_logits = lm_logits[:, -1]

            context, probs, past, generated = self.beam_sample(lm_logits, probs, t, past, generated)

        return generated.cpu()[:, 0], probs

    def beam_sample(self, logits, probs, time_step, past, generated):

        if time_step == 0:
            logits = logits.view(-1, self.beam_size, logits.size()[-1])
            probs, preds = self.beam_start(logits, probs)
            generated = torch.cat([generated, preds], dim=-1)

        else:
            logits = logits.view(-1, self.beam_size, logits.size()[-1])
            probs, preds, past, generated = self.beam_continue(logits, probs, past, generated)

        return preds.view(-1, 1), probs, past, generated

    def beam_start(self, logits, probs):
        logits = logits / self.temperature
        p, i = torch.topk(torch.log_softmax(logits, -1), self.beam_size, -1)  # [batch, beam_size]
        i = i.view(-1, self.beam_size, self.beam_size)[:, 0, :].contiguous().view(-1, 1)
        p = p.view(-1, self.beam_size, self.beam_size)[:, 0, :].contiguous().view(-1, 1)

        probs = probs + p.view(-1)

        return probs, i

    def beam_continue(self, logits, probs, past, generated):
        bs = logits.size(0)
        generated = generated.view(bs, self.beam_size, -1)

        current_p, indexes = torch.topk(torch.log_softmax(logits, -1), self.beam_size,
                                        -1)  # [batch_size, beam_size, beam_size]
        probs = probs.view(bs, -1).unsqueeze(-1) + current_p
        new_probs = probs.view(bs, -1)

        probs, ni = new_probs.topk(self.beam_size, -1)
        sampled = indexes.view(bs, -1).gather(1, ni)  # [batch, beam]
        group = ni // self.beam_size
        ind = torch.arange(bs)[:, None], group
        generated = generated[ind]
        bs_beam = past[0][0].size(0)

        n_head, seq_len, hidden_size = past[0][0].size()[1:]

        past = [
            (k.view(bs, self.beam_size, n_head, seq_len, hidden_size)[ind].view(bs_beam, n_head, seq_len, hidden_size),
             v.view(bs, self.beam_size, n_head, seq_len, hidden_size)[ind].view(bs_beam, n_head, seq_len, hidden_size)) \
            for k, v in past]

        # sampled = indexes.view(bs, -1).gather(1, ni)
        generated = torch.cat([generated, sampled[:, :, None]], -1)

        return probs, sampled.view(-1)[:, None], past, generated


## Huggingface 정답

In [43]:
gen_ids = model.generate(torch.tensor([input_ids]),max_length=34,
                         num_beams=3,temperature=2.0)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 면역력을 높여야 한다.
면역력을 높여야 하는 이유다.
면역력을 높여야 하는 이유다.
면역력을 높여야 하는 이유다.
면역력을 높여


## 비교해보기

In [44]:
sampler=BeamSampler(model,30,3,temperature=2.0)

sampled_ids=sampler.sample(input_ids)[0]

generated = tokenizer.decode(sampled_ids[0,:].tolist())
print(generated) #idontknowidontknow

근육이 커지기 위해서는 무엇보다 면역력을 높여야 한다.
면역력을 높여야 하는 이유다.
면역력을 높여야 하는 이유다.
면역력을 높여야 하는 이유다.
면역력을 높여


# Stochastic-based Decoding
* Top-k sampling

* Top-p sampling

## Hugging face Library

In [45]:
## top-k sampling

gen_ids = model.generate(torch.tensor([input_ids]),max_length=34,
                         do_sample=True,top_k=5,temperature=1.0)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 면역력을 키우는 것이 무엇보다 중요하다.
면역력이란 우리 몸이 스스로 면역력을 키우기 위해 몸속에 있는 각종 면역물질을 스스로 만들어 내는 능력을 말한다.
면역


* top-p sampling

In [46]:
## top-k sampling

gen_ids = model.generate(torch.tensor([input_ids]),max_length=34,
                         do_sample=True,top_p=0.1,temperature=1.0)
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 영양소가 풍부한 음식을 섭취하는 것이 중요하다.
특히, 비타민C가 풍부한 음식은 비타민C가 풍부한 음식을 섭취하는 것이 좋다.
비타민C는


In [47]:
def top_k_logits(logits, k):
    if k == 0:
        # no truncation
        return logits
    else:
        values, _ = torch.topk(logits, k=k)
        min_values = values[:, -1, None]
        return torch.where(
            logits < min_values,
            torch.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )

In [48]:
def top_p_logits(logits, p):
    """Nucleus sampling"""
    batch = logits.size(0)
    sorted_logits, _ = torch.sort(logits, descending=True, dim=-1)
    cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
    a = torch.arange(0, batch).to(logits.device)
    b = torch.max(torch.sum(cumulative_probs <= p, dim=-1) - 1, torch.Tensor([0]).long().to(logits.device))
    min_values = sorted_logits[a, b].to(logits.device)

    return torch.where(
        logits < min_values[:, None],
        torch.ones_like(logits) * -1e10,
        logits,
    )

In [49]:
class StochasticSampler(SamplerBase):
    def __init__(self, model, seq_length, top_whatever, stochastic_func, temperature: float = 1.0):
        """
        :param model:
        :param seq_length:
        :stochastic_func
        """
        super(StochasticSampler, self).__init__(model, seq_length)

        self.temperature = temperature
        self.top_whatever=top_whatever
        self.sampling = stochastic_func
        

    @torch.no_grad()
    def sample(self, inps):
        inps=torch.LongTensor([inps])
        context = inps
        generated = deepcopy(inps)
        past = None
        g_cpu = torch.Generator(device='cpu')
        for t in range(0, self.seq_length):
            out = self.model(context, past_key_values=past)
            lm_logits,past= out["logits"],out["past_key_values"]            
            lm_logits = lm_logits / self.temperature
            lm_logits = lm_logits[:, -1]
            masked_lm_logits = self.sampling(lm_logits, self.top_whatever)
            
            context = torch.multinomial(torch.softmax(masked_lm_logits, -1), 1,generator=g_cpu)
            generated = torch.cat([generated, context], dim=-1)

        return generated


In [50]:
sampler=StochasticSampler(model,30,10,top_k_logits)
sampled_ids=sampler.sample(input_ids)
generated = tokenizer.decode(sampled_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 무엇보다 균형 잡힌 영양 섭취와 충분한 수면을 통해 신체 리듬을 맞추는 게 중요하다.
또한 규칙적인 생활도 중요하지만 무엇보다 중요한 것이 균형 잡힌 식습


In [51]:
sampler=StochasticSampler(model,30,0.5,top_p_logits)
sampled_ids=sampler.sample(input_ids)
generated = tokenizer.decode(sampled_ids[0,:].tolist())
print(generated)

근육이 커지기 위해서는 면역력을 높여야 한다.
면역력을 높이는 가장 좋은 방법은 땀을 많이 흘리는 것이다.
땀을 많이 흘리는 것은 땀을 많이 흘리는 것과 관계가


## 참고 자료

https://huggingface.co/transformers/v2.6.0/quickstart.html#using-the-past

https://github.com/pytorch/fairseq/blob/1f7ef9ed1e1061f8c7f88f8b94c7186834398690/fairseq/search.py#L103

https://jeongukjae.github.io/posts/cs224n-lecture-15-natural-language-generation/